# Heisenberg Uncertainty Principle

This notebook contains the programmatic verification for the **Heisenberg Uncertainty Principle** entry from the THEORIA dataset.

**Entry ID:** heisenberg_uncertainty_principle  
**Required Library:** sympy 1.13.1

## Description
The Heisenberg uncertainty principle states that for any pair of non-commuting observables their statistical spreads (standard deviations) in a given quantum state cannot both be made arbitrarily small. The general Robertson relation ties the product Delta A * Delta B to the expectation value of their commutator, while the canonical position–momentum case gives Delta x * Delta p_x >= hbar/2. This expresses a structural limitation imposed by the quantum operator algebra rather than by experimental imperfections.

## Installation
First, let's install the required library:

In [ ]:
# Install required library with exact version
!pip install sympy==1.13.1

## Programmatic Verification

The following code verifies the derivation mathematically:

In [ ]:
import sympy as sp

# ============================================================
# Programmatic verification for Heisenberg uncertainty entry
# We verify the Robertson relation for a concrete 2D example:
# A = sigma_x, B = sigma_y acting on a qubit Hilbert space.
# This checks the algebraic structure of the derivation.
# ============================================================

# Step 1: Define real parameters for a generic normalized qubit state
theta, phi = sp.symbols('theta phi', real=True)

# |psi> = cos(theta/2)|0> + e^{i phi} sin(theta/2)|1>
psi = sp.Matrix([
    sp.cos(theta/2),
    sp.exp(sp.I*phi) * sp.sin(theta/2)
])

# Step 2: Define Pauli matrices and choose A = sigma_x, B = sigma_y
sigma_x = sp.Matrix([[0, 1], [1, 0]])
sigma_y = sp.Matrix([[0, -sp.I], [sp.I, 0]])
sigma_z = sp.Matrix([[1, 0], [0, -1]])
I2 = sp.eye(2)

A = sigma_x
B = sigma_y

# Check Hermiticity and squares A^2 = B^2 = I (used in derivation)
assert A.H == A
assert B.H == B
assert A**2 == I2
assert B**2 == I2

# Step 3: Compute expectation values a0 = <A>, b0 = <B>
a0 = (psi.H * A * psi)[0]
b0 = (psi.H * B * psi)[0]

a0_s = sp.simplify(a0)
b0_s = sp.simplify(b0)

# Sanity check: expectations are real for Hermitian A,B on normalized psi
assert sp.simplify(a0_s - sp.conjugate(a0_s)) == 0
assert sp.simplify(b0_s - sp.conjugate(b0_s)) == 0

# Step 4: Compute variances Delta A^2 and Delta B^2 via
# Delta A^2 = <(A - a0 I)^2> = <A^2> - 2 a0 <A> + a0^2 <I>
exp_A2 = (psi.H * (A**2) * psi)[0]
exp_B2 = (psi.H * (B**2) * psi)[0]
exp_I = (psi.H * I2 * psi)[0]

DeltaA2 = sp.simplify(exp_A2 - 2*a0_s*a0_s + a0_s**2 * exp_I)
DeltaB2 = sp.simplify(exp_B2 - 2*b0_s*b0_s + b0_s**2 * exp_I)

# For Pauli matrices A^2 = B^2 = I, so <A^2> = <B^2> = 1 and
# Delta A^2 = 1 - a0^2, Delta B^2 = 1 - b0^2.
DeltaA2_expected = sp.simplify(1 - a0_s**2)
DeltaB2_expected = sp.simplify(1 - b0_s**2)

assert sp.simplify(DeltaA2 - DeltaA2_expected) == 0
assert sp.simplify(DeltaB2 - DeltaB2_expected) == 0

# Step 5: Form commutator [A,B] and its expectation value
comm = A*B - B*A
comm_exp = sp.simplify((psi.H * comm * psi)[0])

# For sigma_x, sigma_y one has [sigma_x, sigma_y] = 2 i sigma_z
comm_expected = sp.simplify(2*sp.I * (psi.H * sigma_z * psi)[0])
assert sp.simplify(comm_exp - comm_expected) == 0

# Step 6: Compute LHS and RHS of Robertson inequality
lhs = sp.simplify(DeltaA2_expected * DeltaB2_expected)
rhs = sp.simplify((comm_exp * sp.conjugate(comm_exp)) / 4)

# Verify that lhs - rhs simplifies to a manifestly non-negative expression
diff = sp.simplify(lhs - rhs)

# Sympy yields diff = ((1 - cos(4 phi)) * (1 - cos(2 theta))^2) / 32
diff_expected = (1 - sp.cos(4*phi)) * (1 - sp.cos(2*theta))**2 / 32

assert sp.simplify(diff - diff_expected) == 0

# For real theta, phi we have 1 - cos(4 phi) >= 0 and (1 - cos(2 theta))^2 >= 0,
# hence diff >= 0 and thus
#   Delta A^2 Delta B^2 >= (1/4) |< [A,B] >|^2
# for this example, illustrating the Robertson relation.

# Step 7: Specialization to canonical pair x, p_x.
# Symbolically check that inserting <[x,p_x]> = i hbar into the Robertson bound
# gives the standard Heisenberg inequality Delta x * Delta p_x >= hbar/2.
Dx, Dp, hbar = sp.symbols('Delta_x Delta_p_x hbar', positive=True, real=True)

# Right-hand side from commutator expectation: (1/2) |< [x,p_x] >|
comm_xp = sp.I * hbar   # <[x,p_x]> = i hbar
rhs_xp = sp.simplify(sp.sqrt(comm_xp * sp.conjugate(comm_xp)) / 2)

assert sp.simplify(rhs_xp - hbar/2) == 0

# Encode the squared Heisenberg inequality:
#   (Delta x)^2 (Delta p_x)^2 >= (hbar^2)/4
lhs_xp = Dx**2 * Dp**2
rhs_xp_squared = (hbar**2) / 4

# We cannot prove the inequality symbolically here, but we confirm the algebraic
# form of the lower bound.
expr_form = sp.simplify(rhs_xp_squared - (hbar**2)/4)
assert expr_form == 0

print("Heisenberg / Robertson uncertainty checks passed.")


## Source

📖 **View this entry:** [theoria-dataset.org/entries.html?entry=heisenberg_uncertainty_principle.json](https://theoria-dataset.org/entries.html?entry=heisenberg_uncertainty_principle.json)

This verification code is part of the [THEORIA dataset](https://github.com/theoria-dataset/theoria-dataset), a curated collection of theoretical physics derivations with programmatic verification.

**License:** CC-BY 4.0